In [1]:
import pandas as pd
import os

In [2]:
#路径设置
#银联返回表路径
UnionPayData_Folder=  r"..\1、银联返回表"
#管理端导出路径
SystemData_Folder=  r"..\2、管理端导出表"

In [3]:
#合并csv文件函数
def File_merge_csv(Folder):  #Folder文件夹路径
    File_List = []  #存储文件夹下所有文件的路径
    DataFrame_List = []  #存储dataframe
    for root,dirs,files in os.walk(Folder):
        for file in files:
            file_with_path = os.path.join(root,file)
            File_List.append(file_with_path)
            data_original = pd.read_csv(file_with_path,names=["A"])
            DataFrame_List.append(data_original)
    datas = pd.concat(DataFrame_List,axis=0)
    return datas


#合并excel文件函数
def File_merge_excel(Folder):  #Folder文件夹路径
    File_List = []  #存储文件夹下所有文件的路径
    DataFrame_List = []  #存储dataframe
    for root,dirs,files in os.walk(Folder):
        for file in files:
            file_with_path = os.path.join(root,file)
            File_List.append(file_with_path)
            data = pd.read_excel(file_with_path)
            DataFrame_List.append(data)
    datas = pd.concat(DataFrame_List,axis=0)
    return datas

In [4]:
#读取银联表格和管理端表格
UnionPayData = File_merge_csv(UnionPayData_Folder)
SystemData = File_merge_excel(SystemData_Folder)

In [5]:
#银联表数据清理
#银联表格分列
UnionPayData=UnionPayData.iloc[:,0].str.split("|",expand=True)


In [6]:
#银联表格列重命名
UnionPayData.columns = ["渠道商APPKEY","下单时间","支付单号","渠道商订单号","渠道商订单状态","支付金额","订单金额","充值账号","产品类型","运营商","卡性质","清算金额","行业商户代码","收单机构标识码","发送机构标识码","系统跟踪号","交易传输时间"]
#银联表格删除空值行
UnionPayData.dropna(axis=0,subset=['交易传输时间'],inplace=True)
#银联表格列筛选
#UnionPayData = UnionPayData.loc[:,['支付单号','支付金额','订单金额','充值账号','卡性质','清算金额']]
#管理端表格数据清洗
SystemData.replace("\t","",regex=True,inplace=True)
SystemData = SystemData.rename(columns=lambda x: x.replace("","").replace('\t',''))


In [7]:
#合并文件数据（并集合并）
Outer_Data = pd.merge(UnionPayData,SystemData,left_on="支付单号",right_on="定单号",how="outer")

In [8]:
inner_Data = pd.merge(UnionPayData,SystemData,left_on="支付单号",right_on="定单号",how="inner")

In [9]:
#支付单号为空为系统有记录订单，属于银联多结算账单
UnionPay_outerput = Outer_Data[Outer_Data['支付单号'].isnull()]
UnionPay_outerput.fillna("/",inplace=True)
UnionPay_outerput.to_excel(r'..\3、结果\银联【未】结算账单.xlsx', index=False)

C:\Users\yuhuyi\AppData\Local\Temp\ipykernel_3724\2868406986.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  UnionPay_outerput.fillna("/",inplace=True)


In [10]:
#定单号为空为系统无记录订单，属于银联多结算账单
System_outerput = Outer_Data[Outer_Data['定单号'].isnull()]
System_outerput.fillna("/",inplace=True)
System_outerput.to_excel(r'..\3、结果\银联【多】结算账单.xlsx', index=False)

C:\Users\yuhuyi\AppData\Local\Temp\ipykernel_3724\3606166807.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  System_outerput.fillna("/",inplace=True)


In [11]:
#总数合计校验